In [1]:
import sys
sys.path.append('../src/')
from vi_hrg_approx import *
from utils import c2d, hyperdist, p_hd, polar2cart, warn_tensor
from torch import autograd
torch.manual_seed(4)

In [2]:
def noise_r(x, R, rel_var=0.1, epsilon=1e-4):
    rs = torch.distributions.normal.Normal(x, R*rel_var).sample() 
    return torch.clamp(rs, min=0+epsilon, max=R.item()-epsilon)

In [3]:
def noise_phi(x, rel_var=0.1):
    phis = torch.distributions.normal.Normal(x, 2*np.pi*rel_var).sample()
    return phis % (2*np.pi)

In [4]:
logit = lambda x: (x/(1-x)).log()

In [5]:
N = 75
R = torch.tensor([5.0]).double()
alpha = 1.1
T = 0.1

G = HRG(R=R, alpha=alpha, T=T)
r, theta, A = G.generate(N)
#G.show()
#G.plot()

In [20]:
r_loc_init = logit(noise_r(r, R, rel_var=0.05)/R)*R
r_scale_init = torch.ones([N]).double()
phi_loc_init = torch.cat((polar2cart(1, noise_phi(theta, 0.02)),  torch.zeros([N,1]).double()), dim=-1)
phi_loc_init.shape

torch.Size([75, 3])

In [21]:
R_conc_init = torch.tensor(1.).log()
R_scale_init = torch.tensor(1.).log()
alpha_conc_init = torch.tensor(.5).log()
alpha_scale_init = torch.tensor(.5).log()
T_init = torch.tensor([0.1,10.]).log()

In [22]:
dataloader = DataLoader(EdgesDataset(A), batch_size=128, shuffle=True, num_workers=0)
vi = VI_HRG(75,10000, init_values={'rs_loc':r_loc_init,
                                'rs_scale':r_scale_init,
                              'phis_loc':phi_loc_init,
                              'phis_scale':None, 
                              'R_conc':R_conc_init, 
                              'R_scale':R_scale_init,
                              'alpha_conc':alpha_conc_init,
                              'alpha_scale':alpha_scale_init,
                              'T':T_init})
vi.dataloader = dataloader
vi.optimizer = torch.optim.RMSprop(vi.parameters())
#vi.optimizer.lr

In [23]:
with autograd.detect_anomaly():
    vi.train(dataloader, lrs=0.01, debug=True)

>>>>>>>>>>>> Start training...
-D_kl(R)    >> tensor(-0., dtype=torch.float64, grad_fn=<MulBackward0>)
-D_kl(alpha)>> tensor(-0.0194, dtype=torch.float64, grad_fn=<MulBackward0>)
-D_kl(T)    >> tensor(-0.2115, dtype=torch.float64, grad_fn=<MulBackward0>)
Prob_edges  >> tensor(-883.7948, dtype=torch.float64, grad_fn=<SumBackward0>)
Alpha_R_ri  >> tensor(3.7482, dtype=torch.float64, grad_fn=<MulBackward0>)
Alpha       >> tensor(-4.5663, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor([[13.3929, 11.9329, -0.3919,  ...,  0.9234,  4.2801,  4.4475],
        [12.2177, 11.4401,  1.4114,  ...,  8.5266,  8.4590,  3.6972],
        [12.2964,  9.4241,  0.2273,  ...,  3.3711,  6.3530,  1.3438],
        ...,
        [12.6911,  5.5485,  0.9358,  ...,  8.5466,  7.8834,  2.6422],
        [14.7385, 10.9339,  1.1336,  ...,  8.9757,  2.7591,  4.7790],
        [12.7462, 13.4733,  1.6837,  ...,  7.1029,  7.6619,  8.1885]],
       dtype=torch.float64, grad_fn=<AddBackward0>)
P(q_ri)     >> tensor(-2119.65

P(q_phii)   >> tensor(510.1860, dtype=torch.float64, grad_fn=<MulBackward0>)
ELBO >>>> tensor(-1927.2292, dtype=torch.float64, grad_fn=<AddBackward0>)
-D_kl(R)    >> tensor(-0.0109, dtype=torch.float64, grad_fn=<MulBackward0>)
-D_kl(alpha)>> tensor(-0.0018, dtype=torch.float64, grad_fn=<MulBackward0>)
-D_kl(T)    >> tensor(-0.1331, dtype=torch.float64, grad_fn=<MulBackward0>)
Prob_edges  >> tensor(-773.5014, dtype=torch.float64, grad_fn=<SumBackward0>)
Alpha_R_ri  >> tensor(33.7194, dtype=torch.float64, grad_fn=<MulBackward0>)
Alpha       >> tensor(-1.8619, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor([[ 7.5559e+00,  9.7224e+00, -1.4205e+00,  ...,  3.0385e+00,
         -4.6941e+00,  6.0109e+00],
        [ 9.6768e+00,  9.6367e+00,  1.0617e+00,  ...,  3.1672e+00,
          4.6296e+00,  2.7070e+00],
        [ 1.1581e+01,  1.3035e+01,  3.1744e-01,  ...,  2.7933e+00,
          6.6206e+00,  1.5366e+00],
        ...,
        [ 6.1093e+00,  1.1839e+01,  1.3839e+00,  ...,  1.2737e+01,
  

P(q_phii)   >> tensor(526.9343, dtype=torch.float64, grad_fn=<MulBackward0>)
ELBO >>>> tensor(-1081.0329, dtype=torch.float64, grad_fn=<AddBackward0>)
-D_kl(R)    >> tensor(-0.0349, dtype=torch.float64, grad_fn=<MulBackward0>)
-D_kl(alpha)>> tensor(-0.0025, dtype=torch.float64, grad_fn=<MulBackward0>)
-D_kl(T)    >> tensor(-0.1010, dtype=torch.float64, grad_fn=<MulBackward0>)
Prob_edges  >> tensor(-659.9139, dtype=torch.float64, grad_fn=<SumBackward0>)
Alpha_R_ri  >> tensor(168.0976, dtype=torch.float64, grad_fn=<MulBackward0>)
Alpha       >> tensor(-0.0844, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor([[ 9.4040,  7.7538, -1.3927,  ..., -3.2508,  6.1879,  5.8838],
        [ 9.6582,  8.7013, -1.3256,  ...,  7.2311,  3.3778,  2.2881],
        [13.0867, 15.7494, -0.5207,  ..., -1.7061,  4.2114,  2.7378],
        ...,
        [13.6690,  4.3658, -1.9074,  ...,  3.9962,  4.8387, -3.2670],
        [11.2060, 13.5517, -1.1609,  ...,  3.5244,  1.1376,  1.7601],
        [ 7.7494,  0.6175, 

-D_kl(R)    >> tensor(-0.0937, dtype=torch.float64, grad_fn=<MulBackward0>)
-D_kl(alpha)>> tensor(-0.0314, dtype=torch.float64, grad_fn=<MulBackward0>)
-D_kl(T)    >> tensor(-0.0818, dtype=torch.float64, grad_fn=<MulBackward0>)
Prob_edges  >> tensor(-593.9184, dtype=torch.float64, grad_fn=<SumBackward0>)
Alpha_R_ri  >> tensor(629.1786, dtype=torch.float64, grad_fn=<MulBackward0>)
Alpha       >> tensor(1.6712, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor([[15.8063,  8.7604, -0.2983,  ...,  9.4373,  2.8545,  1.8848],
        [ 7.4858,  2.5660, -1.8504,  ...,  4.4903,  3.6284,  2.7210],
        [10.5366, 10.9851, -2.7073,  ...,  6.8032, -4.7294,  0.8651],
        ...,
        [ 0.7726, 10.2239, -2.4984,  ...,  3.1410,  6.3813, -3.7102],
        [ 8.3315, 13.5217, -1.8214,  ..., 10.8119, -3.3415,  4.8381],
        [11.8188,  6.1284, -1.0341,  ...,  4.0301,  6.0216, -2.0191]],
       dtype=torch.float64, grad_fn=<AddBackward0>)
P(q_ri)     >> tensor(516.0833, dtype=torch.float64, gra

-D_kl(R)    >> tensor(-0.2209, dtype=torch.float64, grad_fn=<MulBackward0>)
-D_kl(alpha)>> tensor(-0.1224, dtype=torch.float64, grad_fn=<MulBackward0>)
-D_kl(T)    >> tensor(-0.0692, dtype=torch.float64, grad_fn=<MulBackward0>)
Prob_edges  >> tensor(-463.1278, dtype=torch.float64, grad_fn=<SumBackward0>)
Alpha_R_ri  >> tensor(3862.3044, dtype=torch.float64, grad_fn=<MulBackward0>)
Alpha       >> tensor(3.2735, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor([[ 7.5746, 16.0691, -1.3693,  ..., -3.6246,  2.0164, -0.5557],
        [12.6706,  4.8469, -2.0531,  ..., -3.6176,  3.5550,  4.4676],
        [ 6.3327, 11.5017, -1.6287,  ..., -1.8939, -1.2536,  3.4610],
        ...,
        [ 2.1847, 13.2198, -1.6295,  ...,  0.3601,  9.8354, -2.9795],
        [10.2129,  0.4921, -0.5223,  ...,  4.5894,  0.8705, -1.0953],
        [ 9.7177, 14.7346, -1.6588,  ..., -2.5876, -1.5461, -3.6364]],
       dtype=torch.float64, grad_fn=<AddBackward0>)
P(q_ri)     >> tensor(1981.7410, dtype=torch.float64, g

-D_kl(R)    >> tensor(-0.6215, dtype=torch.float64, grad_fn=<MulBackward0>)
-D_kl(alpha)>> tensor(-0.3880, dtype=torch.float64, grad_fn=<MulBackward0>)
-D_kl(T)    >> tensor(-0.0597, dtype=torch.float64, grad_fn=<MulBackward0>)
Prob_edges  >> tensor(-526.6494, dtype=torch.float64, grad_fn=<SumBackward0>)
Alpha_R_ri  >> tensor(31146.9242, dtype=torch.float64, grad_fn=<MulBackward0>)
Alpha       >> tensor(4.9851, dtype=torch.float64, grad_fn=<MulBackward0>)
tensor([[ 2.0161, -3.9873, -3.6666,  ...,  0.7459, -3.8468, -0.8289],
        [ 7.1009, -5.8829, -3.8497,  ..., 10.9264,  4.9657, -3.7455],
        [ 2.3546, -5.0743, -3.3526,  ..., -5.0380,  1.0675,  2.4617],
        ...,
        [ 9.2452, 12.9777, -3.2167,  ...,  5.3055, 13.4107, -5.0698],
        [ 0.7355,  5.3339, -4.0537,  ..., -3.9400, -5.7877, -3.8963],
        [-3.6509, -1.7134, -2.9542,  ..., -3.4289, -2.9431, -3.5648]],
       dtype=torch.float64, grad_fn=<AddBackward0>)
P(q_ri)     >> tensor(6660.0216, dtype=torch.float64, 

RuntimeError: Function 'DivBackward0' returned nan values in its 1th output.

In [12]:
vi.alpha_conc.exp(), vi.alpha_scale.exp()

(tensor(7.4369, dtype=torch.float64, grad_fn=<ExpBackward>),
 tensor(7.3587, dtype=torch.float64, grad_fn=<ExpBackward>))

In [13]:
vi.rs_loc, vi.rs_scale.exp()

(Parameter containing:
 tensor([  9.8982,  10.0683,   1.0266,   3.0196,  -3.8415,  -2.4633,  17.1828,
          13.8425,   9.1261,  10.8953,   6.0872,   2.6963,  55.5907,  10.3392,
          23.8557,  18.1783,  13.5867,   1.5954,   4.2125,  -1.6708,  12.2500,
           3.4677,   7.2140,   6.1196,  13.6078,  19.9534,  55.5108,  55.6662,
          22.5025,   3.8846,  55.4761,  14.6449,   3.3682,  55.6548,  11.3391,
           3.1483,   3.5842,  17.7127,  55.5192,  11.3821,  13.6212,  11.7332,
          55.6028,   2.5591,   4.8713,  20.8397,  12.8007,   5.5934,   5.0820,
          13.4547,   3.9258,  11.8415,  16.2995,  -3.4724, -52.9392,  11.1797,
          11.3130,  31.1252,  12.9965,  16.8723,   2.7077,  27.4110,  -5.9850,
          20.3944,   9.2604,  10.3816,   8.0351,   2.1037,  10.4496,   8.3832,
          12.1950,  -0.4643,   3.7975,   3.2438,   2.0786], dtype=torch.float64,
        requires_grad=True),
 tensor([10.5677, 12.0597,  7.9715,  8.0311,  1.0977,  0.8675,  9.4963,  6.34

In [14]:
post_r = Radius(vi.rs_loc, vi.rs_scale.exp(), torch.ones([N])*R).mean

In [15]:
post_r - r

tensor([ 0.5216,  0.5226,  0.7863,  1.1588, -1.0225, -1.8175,  0.0064,  0.0819,
         0.4316,  0.0233,  1.0120,  1.0557,  0.0690,  0.6713,  0.4671,  0.2060,
         0.2430,  0.5331,  0.9852, -1.4102,  0.5842,  0.8670,  0.4448,  0.5711,
         0.0690,  0.5488,  0.2584,  0.3770,  0.0523,  1.0003,  0.0776,  0.1373,
         0.9097,  0.1937,  0.0346,  1.1559,  0.9279,  0.1549,  0.0918,  0.4129,
         0.3828,  0.1251,  0.0575,  1.1490,  1.0563,  0.0742,  0.2353,  1.1673,
         0.9613,  0.0134,  1.3969,  0.1770,  0.2122, -1.5952, -0.4163,  0.3658,
         0.2646,  0.0527,  0.5187,  0.1579,  1.4028,  0.2027, -1.6590,  0.1725,
         0.5302,  0.1910,  0.3227,  1.1171,  0.6134,  0.5150,  0.5025, -0.6480,
         1.1378,  1.0011,  0.7626], dtype=torch.float64,
       grad_fn=<SubBackward0>)

In [ ]:
Radius(vi.rs_loc, vi.rs_scale.exp(), torch.ones([N])*R).entropy()

In [ ]:
vi.r_i_samples

In [ ]:
vi.R_conc.exp(), vi.R_scale.exp()

In [ ]:
p_hd_ = lambda d,R,T: (1.+((d-R)/(2.*T)).exp()).reciprocal()
phd = lambda d,R,T: 0.5 + 0.5*(-(d-R)/(4*T)).tanh()

In [ ]:
p_hd_(torch.tensor(-np.inf),R,T)

In [ ]:
phd(torch.tensor(5.1),R,T)

In [ ]:
torch.tensor([0.1,10.0]).log()

In [ ]:
x = torch.arange(0.,1.,.01)
plt.plot(x, torch.sigmoid(logit(x)))
plt.show()

In [ ]:
logit(torch.tensor(0.))

In [ ]:
l, n, m = 3, 4, 5
broadcast = torch.ones([l,n])

In [ ]:
torch.rand(m).expand(n,m).t()

In [ ]:
torch.arange(n).expand(m,n)

In [ ]:
torch.tensor(0.).mean()

In [ ]:
arcosh = lambda x: (torch.clamp(x, min=1.) + (torch.clamp(x, min=1.)**2 - 1).sqrt())
arcosh(torch.arange(0.,2., .1))

In [ ]:
torch.clamp(x, min=1.)

In [ ]:
np.log(5)